# Evaluation part I

Evaluate LLM responses when there is a single "right answer".

After you've built such a system, how do you know how it's working? 

And maybe even as you deploy it and let users use it, 
how can you track how it's doing and find any 
shortcomings and continue to improve the quality of 
the answers of your system? 

In this video, I'd like to share with you some best practices 
for __evaluating the outputs of an LLM__ and I want to share 
with you specifically what it feels like to build one of these 
systems. 
One key distinction between what you hear me talk 
about in this video and what you may have seen in more 
traditional machine learning supervised learning applications 
is because you can build such an application so quickly, 
the methods for evaluating it, it tends not to start 
off with a test set. Instead, you often __end up 
gradually building up a set of test examples.__

![Token](immagini\09_evaluation.png)

In the traditional supervised learning approach, collecting additional test examples was manageable due to the substantial size of the training data. This involved dividing the data into training, development, and test sets, commonly used during the development process.

However, building applications using __modern language models like LLMs follows a different approach__. Developers start with a __small set__ of labeled examples and __fine-tune__ the model by tuning prompts to work effectively on this limited data.

As testing and evaluation progress, __more examples are opportunistically added to the development set to create a slowly growing evaluation data__. Metrics like average accuracy are used to measure performance on this small set of examples, and developers have the flexibility to stop if the model meets their requirements at any point.

__If the confidence in the model's performance is still insufficient, a randomly sampled set of examples is collected for further tuning__, which serves as an extended development set. Only if a higher level of precision is needed, such as improving performance from 91% to 93%, is a hold-out test set utilized.

For high-stakes applications where incorrect outputs can cause harm, __it is crucial to rigorously evaluate the system's performance__. In such cases, both development and test sets are necessary to ensure correctness and safety.

For lower-risk tasks, developers can stop the process earlier if the risks are modest, avoiding the effort of collecting larger datasets for evaluation.

__In summary, the process of building applications using LLMs involves starting with a small set of examples, gradually expanding the development set, and using metrics to measure performance. The evaluation process can be stopped if the model meets requirements, but for high-stakes applications, rigorous evaluation with test sets becomes essential.__

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [ ]:
import os
import openai
import sys
sys.path.append('../..')
import utils
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

#### Get the relevant products and categories
Here is the list of products and categories that are in the product catalog.

In [ ]:
products_and_category = utils.get_products_and_category()
products_and_category
# Use the utils function to get a list of products and categories. 

*OUTPUT*
```json
{'Computers and Laptops': ['TechPro Ultrabook',
  'BlueWave Gaming Laptop',
  'PowerLite Convertible',
  'TechPro Desktop',
  'BlueWave Chromebook'],
 'Smartphones and Accessories': ['SmartX ProPhone',
  'MobiTech PowerCase',
  'SmartX MiniPhone',
  'MobiTech Wireless Charger',
  'SmartX EarBuds'],
 'Televisions and Home Theater Systems': ['CineView 4K TV',
  'SoundMax Home Theater',
  'CineView 8K TV',
  'SoundMax Soundbar',
  'CineView OLED TV'],
 'Gaming Consoles and Accessories': ['GameSphere X',
  'ProGamer Controller',
  'GameSphere Y',
  'ProGamer Racing Wheel',
  'GameSphere VR Headset'],
 'Audio Equipment': ['AudioPhonic Noise-Canceling Headphones',
  'WaveSound Bluetooth Speaker',
  'AudioPhonic True Wireless Earbuds',
  'WaveSound Soundbar',
  'AudioPhonic Turntable'],
 'Cameras and Camcorders': ['FotoSnap DSLR Camera',
  'ActionCam 4K',
  'FotoSnap Mirrorless Camera',
  'ZoomMaster Camcorder',
  'FotoSnap Instant Camera']}
  ```

### Find relevant product and category names (version 1)
This could be the version that is running in production.

In [ ]:
def find_category_and_product_v1(user_input,products_and_category):

    delimiter = "####"
    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be delimited with {delimiter} characters.
    Output a python list of json objects, where each object has the following format:
        'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    AND
        'products': <a list of products that must be found in the allowed products below>


    Where the categories and products must be found in the customer service query.
    If a product is mentioned, it must be associated with the correct category in the allowed products list below.
    If no products or categories are found, output an empty list.
    

    List out all products that are relevant to the customer service query based on how closely it relates
    to the product name and product category.
    Do not assume, from the name of the product, any features or attributes such as relative quality or price.

    The allowed products are provided in JSON format.
    The keys of each item represent the category.
    The values of each item is a list of products that are within that category.
    Allowed products: {products_and_category}
    

    """
    
    few_shot_user_1 = """I want the most expensive computer."""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)


few_shot

But the prompt specifies a set of instructions, and 
it actually gives the language model one example 
of a good output. This is sometimes called a few-shot or 
technically one-shot prompting, because we're actually using a 
user message and a system message to give 
it one example of a good output.

### Evaluate on some queries

In [ ]:
customer_msg_0 = f"""Which TV can I buy if I'm on a budget?"""

products_by_category_0 = find_category_and_product_v1(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

*OUTPUT*
```json
[{'category': 'Televisions and Home Theater Systems',
  'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]
```

In [ ]:
customer_msg_1 = f"""I need a charger for my smartphone"""

products_by_category_1 = find_category_and_product_v1(customer_msg_1,
                                                      products_and_category)
print(products_by_category_1)

*OUTPUT*
```json
[{'category': 'Smartphones and Accessories', 
  'products': ['MobiTech PowerCase', 'MobiTech Wireless Charger', 'SmartX EarBuds']}]
```

In [ ]:
customer_msg_2 = f"""
What computers do you have?"""

products_by_category_2 = find_category_and_product_v1(customer_msg_2,
                                                      products_and_category)
products_by_category_2

*OUTPUT*
```json
" \n    
[{'category': 'Computers and Laptops', 
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]"
```

So, here I have three prompts, and if you are developing 
this prompt for the first time, it would be quite reasonable 
to to have one or two or three examples like this, 
and to keep on tuning the prompt until 
it gives appropriate outputs, until the prompt is retrieving the 
relevant products and categories to the 
customer request for all of your prompts, 
all three of them in this example. And if the prompt had been 
missing some products or something, then what we would 
do is probably go back to edit the prompt a few times until it gets it 
right on all three of these prompts. 
After you've gotten the system to this point, you 
might then start running the system in testing. Maybe 
send it to internal test users or try 
using it yourself, and just run it for a 
while to see what happens. 
And sometimes you will run across a prompt that it fails on. 

In [ ]:
customer_msg_3 = f"""
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs do you have?"""

products_by_category_3 = find_category_and_product_v1(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

*OUTPUT*
![Token](immagini\10_output.png)

So when I run it on this prompt, it looks like it's 
outputting the right data, but it also outputs a bunch of text here, 
this extra junk. It makes it harder to 
parse this into a Python list of dictionaries. 
So we don't like that it's outputting this extra junk. So when 
you run across one example that the system 
fails on, then common practice is to just note down 
that this is a somewhat tricky example, so let's add this 
to our set of examples that we're going to test 
the system on systematically. 
And if you keep on running the system for a while longer, 
maybe it works on those examples. We did tune the 
prompt to three examples, so maybe it will work on many examples, 
but just by chance you might run across another example where it 
generates an error.

### Harder test cases
Identify queries found in production, where the model is not working as expected.

In [ ]:
customer_msg_4 = f"""
tell me about the CineView TV, the 8K one, Gamesphere console, the X one.
I'm on a budget, what computers do you have?"""

products_by_category_4 = find_category_and_product_v1(customer_msg_4,
                                                      products_and_category)
print(products_by_category_4)

*OUTPUT*

![Token](immagini\11_output.png)

So this custom message for also causes the 
system to output a bunch of junk text at the end that 
we don't want. 

And so at this point, you may have run this prompt, 
maybe on hundreds of examples, maybe you have test users, 
but you would just take the examples, the tricky ones is doing poorly on, 
and now I have this set of five examples, index from 0 to 4, 
have this set of five examples that you 
use to further fine-tune the prompts. 

And in both of these examples, the LLM had output a 
bunch of extra junk text at the end that we don't want. 
And after a little bit of trial and error, you might decide to 
modify the prompts as follows. 

### Modify the prompt to work on the hard test cases

In [ ]:
def find_category_and_product_v2(user_input,products_and_category):
    """
    Added: Do not output any additional text that is not in JSON format.
    Added a second example (for few-shot prompting) where user asks for 
    the cheapest computer. In both few-shot examples, the shown response 
    is the full list of products in JSON only.
    """
    delimiter = "####"
    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be delimited with {delimiter} characters.
    Output a python list of json objects, where each object has the following format:
        'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    AND
        'products': <a list of products that must be found in the allowed products below>
    Do not output any additional text that is not in JSON format.
    Do not write any explanatory text after outputting the requested JSON.


    Where the categories and products must be found in the customer service query.
    If a product is mentioned, it must be associated with the correct category in the allowed products list below.
    If no products or categories are found, output an empty list.
    

    List out all products that are relevant to the customer service query based on how closely it relates
    to the product name and product category.
    Do not assume, from the name of the product, any features or attributes such as relative quality or price.

    The allowed products are provided in JSON format.
    The keys of each item represent the category.
    The values of each item is a list of products that are within that category.
    Allowed products: {products_and_category}
    

    """
    
    few_shot_user_1 = """I want the most expensive computer. What do you recommend?"""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    few_shot_user_2 = """I want the most cheapest computer. What do you recommend?"""
    few_shot_assistant_2 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{few_shot_user_2}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_2 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)


### Evaluate the modified prompt on the hard tests cases

In [ ]:
customer_msg_3 = f"""
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs do you have?"""

products_by_category_3 = find_category_and_product_v2(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

*OUTPUT*
```json
[{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]
```

### Regression testing: verify that the model still works on previous test cases
Check that modifying the model to fix the hard test cases does not negatively affect its performance on previous test cases.

In [ ]:
customer_msg_0 = f"""Which TV can I buy if I'm on a budget?"""

products_by_category_0 = find_category_and_product_v2(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

*OUTPUT*
```json
[{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]
```

You can kind of tell that if I had to copy-paste 5 prompts, 
customers such as 0, 1, 2, 3, and 4, into 
my Jupyter notebook and run them and then manually look 
at them to see if they output in the right categories and 
products. You can kind of do it. I can look 
at this and go, "Yep, category, TV and home 
theater systems, products. Yep, looks like you 
got all of them.". 
But it's actually a little bit painful to do this manually, to 
manually inspect or to look at this output to make sure with 
your eyes that this is exactly the right output. 
So when the development set that you're tuning to becomes more 
than just a small handful of examples, it 
then becomes useful to start to automate the testing process. 

### Gather development set for automated testing

In [ ]:
msg_ideal_pairs_set = [
    
    # eg 0
    {'customer_msg':"""Which TV can I buy if I'm on a budget?""",
     'ideal_answer':{
        'Televisions and Home Theater Systems':set(
            ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']
        )}
    },

    # eg 1
    {'customer_msg':"""I need a charger for my smartphone""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['MobiTech PowerCase', 'MobiTech Wireless Charger', 'SmartX EarBuds']
        )}
    },
    # eg 2
    {'customer_msg':f"""What computers do you have?""",
     'ideal_answer':{
           'Computers and Laptops':set(
               ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'
               ])
                }
    },

    # eg 3
    {'customer_msg':f"""tell me about the smartx pro phone and \
    the fotosnap camera, the dslr one.\
    Also, what TVs do you have?""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['SmartX ProPhone']),
        'Cameras and Camcorders':set(
            ['FotoSnap DSLR Camera']),
        'Televisions and Home Theater Systems':set(
            ['CineView 4K TV', 'SoundMax Home Theater','CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV'])
        }
    }, 
    
    # eg 4
    {'customer_msg':"""tell me about the CineView TV, the 8K one, Gamesphere console, the X one.
I'm on a budget, what computers do you have?""",
     'ideal_answer':{
        'Televisions and Home Theater Systems':set(
            ['CineView 8K TV']),
        'Gaming Consoles and Accessories':set(
            ['GameSphere X']),
        'Computers and Laptops':set(
            ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'])
        }
    },
    
    # eg 5
    {'customer_msg':f"""What smartphones do you have?""",
     'ideal_answer':{
           'Smartphones and Accessories':set(
               ['SmartX ProPhone', 'MobiTech PowerCase', 'SmartX MiniPhone', 'MobiTech Wireless Charger', 'SmartX EarBuds'
               ])
                    }
    },
    # eg 6
    {'customer_msg':f"""I'm on a budget.  Can you recommend some smartphones to me?""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['SmartX EarBuds', 'SmartX MiniPhone', 'MobiTech PowerCase', 'SmartX ProPhone', 'MobiTech Wireless Charger']
        )}
    },

    # eg 7 # this will output a subset of the ideal answer
    {'customer_msg':f"""What Gaming consoles would be good for my friend who is into racing games?""",
     'ideal_answer':{
        'Gaming Consoles and Accessories':set([
            'GameSphere X',
            'ProGamer Controller',
            'GameSphere Y',
            'ProGamer Racing Wheel',
            'GameSphere VR Headset'
     ])}
    },
    # eg 8
    {'customer_msg':f"""What could be a good present for my videographer friend?""",
     'ideal_answer': {
        'Cameras and Camcorders':set([
        'FotoSnap DSLR Camera', 'ActionCam 4K', 'FotoSnap Mirrorless Camera', 'ZoomMaster Camcorder', 'FotoSnap Instant Camera'
        ])}
    },
    
    # eg 9
    {'customer_msg':f"""I would like a hot tub time machine.""",
     'ideal_answer': []
    }
    
]


So here is a set of 10 examples where I'm specifying 10 customer messages. So 
here's a customer message, "Which TV can I buy if I'm on a 
budget?" as well as what's the ideal answer.

### Evaluate test cases by comparing to the ideal answers

If you want to evaluate automatically, 
what the prompt is doing on any of these 10 examples, 
here is a function to do so.

In [ ]:
import json

def eval_response_with_ideal(response, ideal, debug=False):
    """
    Evaluta la correttezza delle risposte rispetto alle risposte ideali.
    
    Args:
        response (str): La risposta da valutare (JSON-like).
        ideal (dict): Le risposte ideali strutturate come un dizionario.
        debug (bool, optional): Abilita la modalità di debug per stampare informazioni aggiuntive. Default: False.
    
    Returns:
        float: Percentuale di risposte corrette rispetto alle risposte ideali.
    """
    
    if debug:
        print("response")
        print(response)
    
    # json.loads() si aspetta le doppie virgolette, non gli apici singoli
    json_like_str = response.replace("'", '"')
    
    # Parsing della stringa JSON-like in una lista di dizionari
    l_of_d = json.loads(json_like_str)
    
    # Caso speciale: entrambe le risposte sono vuote
    if l_of_d == [] and ideal == []:
        return 1
    
    # Altrimenti, se la risposta o l'ideale sono vuoti, c'è un mismatch
    elif l_of_d == [] or ideal == []:
        return 0
    
    correct = 0
    
    if debug:
        print("l_of_d is")
        print(l_of_d)
        
    # Iterazione attraverso ciascun dizionario nella lista
    for d in l_of_d:
        cat = d.get('category')
        prod_l = d.get('products')
        
        # Verifica se sia presente la categoria e la lista di prodotti
        if cat and prod_l:
            # Converte la lista in un set per facilitare il confronto
            prod_set = set(prod_l)
            # Ottiene l'insieme ideale di prodotti per la categoria
            ideal_cat = ideal.get(cat)
            
            if ideal_cat:
                prod_set_ideal = set(ideal.get(cat))
            else:
                if debug:
                    print(f"Categoria {cat} non trovata negli ideali")
                    print(f"Ideali: {ideal}")
                continue
                
            if debug:
                print("prod_set\n", prod_set)
                print()
                print("prod_set_ideal\n", prod_set_ideal)

            # Confronto tra insiemi
            if prod_set == prod_set_ideal:
                if debug:
                    print("corretto")
                correct += 1
            else:
                print("errato")
                print(f"prod_set: {prod_set}")
                print(f"prod_set_ideal: {prod_set_ideal}")
                if prod_set <= prod_set_ideal:
                    print("La risposta è un sottoinsieme della risposta ideale")
                elif prod_set >= prod_set_ideal:
                    print("La risposta è un sovrainsieme della risposta ideale")

    # Calcolo percentuale corretta rispetto al numero totale di elementi nella lista
    pc_correct = correct / len(l_of_d)
        
    return pc_correct


In [ ]:
print(f'Customer message: {msg_ideal_pairs_set[7]["customer_msg"]}')
print(f'Ideal answer: {msg_ideal_pairs_set[7]["ideal_answer"]}')

*OUTPUT*

Customer message: What Gaming consoles would be good for my friend who is into racing games?

Ideal answer: {'Gaming Consoles and Accessories': {'GameSphere X', 'GameSphere Y', 'ProGamer Racing Wheel', 'ProGamer Controller', 'GameSphere VR Headset'}}

In [ ]:
response = find_category_and_product_v2(msg_ideal_pairs_set[7]["customer_msg"],
                                         products_and_category)
print(f'Response: {response}')

eval_response_with_ideal(response,
                              msg_ideal_pairs_set[7]["ideal_answer"])

*OUTPUT*

Response:  
    [{'category': 'Gaming Consoles and Accessories', 'products': ['GameSphere X', 'ProGamer Controller', 'GameSphere Y', 'ProGamer Racing Wheel', 'GameSphere VR Headset']}]
    
1.0

### Run evaluation on all test cases and calculate the fraction of cases that are correct

So what I would do if I'm tuning the prompt now is I would then 
use a fold to loop over all 10 of the development set examples, where 
we repeatedly pull out the customer message, get 
the ideal answer, the right answer, call 
the arm to get a response, evaluate it, and then, you know, accumulate 
it in average. 

In [ ]:
# Note, this will not work if any of the api calls time out
score_accum = 0
for i, pair in enumerate(msg_ideal_pairs_set):
    print(f"example {i}")
    
    customer_msg = pair['customer_msg']
    ideal = pair['ideal_answer']
    
    # print("Customer message",customer_msg)
    # print("ideal:",ideal)
    response = find_category_and_product_v2(customer_msg,
                                                      products_and_category)

    
    # print("products_by_category",products_by_category)
    score = eval_response_with_ideal(response,ideal,debug=False)
    print(f"{i}: {score}")
    score_accum += score
    

n_examples = len(msg_ideal_pairs_set)
fraction_correct = score_accum / n_examples
print(f"Fraction correct out of {n_examples}: {fraction_correct}")

*OUTPUT*

example 0

0: 1.0

example 1

incorrect

prod_set: {'SmartX MiniPhone', 'SmartX ProPhone', 'MobiTech Wireless Charger', 'SmartX EarBuds', 'MobiTech PowerCase'}

prod_set_ideal: {'MobiTech Wireless Charger', 'SmartX EarBuds', 'MobiTech PowerCase'}

response is a superset of the ideal answer

1: 0.0

example 2

2: 1.0

example 3

3: 1.0

example 4

incorrect

prod_set: {'SoundMax Soundbar', 'CineView 4K TV', 'SoundMax Home Theater', 'CineView OLED TV', 'CineView 8K TV'}

prod_set_ideal: {'CineView 8K TV'}

response is a superset of the ideal answer

incorrect

prod_set: {'GameSphere X', 'GameSphere Y', 'ProGamer Racing Wheel', 'ProGamer Controller', 'GameSphere VR Headset'}

prod_set_ideal: {'GameSphere X'}

response is a superset of the ideal answer

4: 0.3333333333333333

example 5

5: 1.0

example 6

6: 1.0

example 7

7: 1.0

example 8

8: 1.0

example 9

9: 1

Fraction correct out of 10: 0.8333333333333334

The notebook demonstrates going through steps 1, 2, and 3 of the bulleted list, resulting in a good development set of 10 examples to tune and validate the prompts. For increased rigor, developers can collect a randomly sampled set of around 100 examples, or even go further with a holdout test set for evaluation.

For many applications, stopping at step 3 suffices, as it yields a reasonably performing system quickly. However, for safety-critical applications or those with potential harm, a more extensive test set is necessary before deployment.

The workflow of building applications using prompts is different from traditional supervised learning, with faster iteration. Despite using a small number of hand-curated examples, the evaluation method can be surprisingly effective in improving prompt effectiveness and system performance.